In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from functools import partial, reduce
from matplotlib.patches import Patch

In [ ]:
directory = os.path.abspath(r'Test_dataset_depth') 

In [ ]:
os.chdir(directory)

## Get file names

In [ ]:
def getFiles(directory, suffix1):
    """Create list of files and sample names"""
    fileList = []
    nameList = []
    for f in os.listdir(os.path.abspath(directory)):
        if (f.endswith(suffix1)):
            fileList.append(f)
            fns = f.split(suffix1)[0]
            nameList.append(fns)
    return fileList, nameList

In [ ]:
fileList, nameList = getFiles(directory,'_250000_depth.txt')

## Create dictionary of dataframes from depth profiles

In [ ]:
def makedict(fileList,nameList,directory):
    """Convert depth profiles to dictionary of dfs"""
    dictdf = {}
    renamedict = {}
    
    for i in range(len(fileList)):
        dictdf[nameList[i]] = pd.read_csv(os.path.abspath(directory) +'/'+ fileList[i], sep='\t', header=None, 
                                          names=['REF','POS','Depth'])
   
    for key, df in dictdf.items():
        dfrename = df.rename(columns = {'Depth':key})
        renamedict[key] = dfrename
        
    return renamedict        

In [ ]:
renamedict = makedict(fileList,nameList,directory)

## Merge dataframes in dictionary and subset genome by contig, gene, plasmid, etc.

In [ ]:
def mergedictdf(renamedict,genomeID=""):
    """Merge dataframes in dictionary to dataframe"""   
    my_reduce = partial(pd.merge, on= ['REF','POS'], how='outer')
    mergeddf = reduce(my_reduce,renamedict.values())
    
    if not genomeID:
        genomedf = mergeddf
    else:
        genomedf = mergeddf[mergeddf.REF == genomeID] 
    
    return genomedf

In [ ]:
genomedf = mergedictdf(renamedict,"NC_003197.2")

In [ ]:
genomedf.head()

### Custom sort columns of dataframe (Optional)

In [ ]:
def sortcolumns(genomedf,*args):
    """sort columns in dataframe"""
    if not args:
        df = genomedf
    else:
        tupleorder = args
        newlist = list(tupleorder)
        df = genomedf[genomedf.columns[newlist]]

    return df

In [ ]:
df = sortcolumns(genomedf)

## Set plot labels and colors

In [ ]:
label0 = [Patch(facecolor="forestgreen", label="$10^3$_90.0_(0.001)")]
label1 = [Patch(facecolor="olivedrab", label="$10^3$_50.0_(0.01)")]
label2 = [Patch(facecolor="yellowgreen", label="$10^3$_9.1_(0.1)")] 
label3 = [Patch(facecolor="slategrey", label="$10^3$_1.0_(1)")]

In [ ]:
legend0 = [Patch(facecolor="None", label="cov=61.2%, CV=0.73")]
legend1 = [Patch(facecolor="None", label="cov=17.3%, CV=0.85")]
legend2 = [Patch(facecolor="None", label="cov=2.2%, CV=2.12")]
legend3 = [Patch(facecolor="None", label="cov=0.29%, CV=6.24")]

## Plot

In [ ]:
fig, ax=plt.subplots(4, figsize=(15,20), sharey=True, sharex=True)

#### Plot 0 (label 10)

ax[0].plot(df[df.columns[1]],df[df.columns[2]],color="forestgreen")
ax[0].set_ylim((10**0,10**3))
ax[0].set_yscale("log")
ax[0].legend(handles=label0, loc='upper left', fontsize=18)
ax[0].tick_params(axis='both', which='major', labelsize=14)

axA=ax[0].twinx()
axA.get_yaxis().set_visible(False)
axA.legend(handles=legend0, loc='upper right', fontsize=14, frameon=False)

#### Plot 1

ax[1].plot(df[df.columns[1]],df[df.columns[3]],color="olivedrab") 
ax[1].legend(handles=label1, loc='upper left', fontsize=18)
ax[1].tick_params(axis='both', which='major', labelsize=14)

axB=ax[1].twinx()
axB.get_yaxis().set_visible(False)
axB.legend(handles=legend1, loc='upper right', fontsize=14, frameon=False)

#### Plot 2

ax[2].plot(df[df.columns[1]],df[df.columns[4]],color="yellowgreen") 
ax[2].legend(handles=label2, loc='upper left', fontsize=18)
ax[2].tick_params(axis='both', which='major', labelsize=14)

axC=ax[2].twinx()
axC.get_yaxis().set_visible(False)
axC.legend(handles=legend2, loc='upper right', fontsize=14, frameon=False)

#### Plot 3

ax[3].plot(df[df.columns[1]],df[df.columns[5]],color="slategrey") #turquois blue
ax[3].legend(handles=label3, loc='upper left', fontsize=18)
ax[3].tick_params(axis='both', which='major', labelsize=14)

axD=ax[3].twinx()
axD.get_yaxis().set_visible(False)
axD.legend(handles=legend3, loc='upper right', fontsize=14, frameon=False)

#### Figure parameters

fig.text(0.05, 0.5, "Read Depth (Log$_{10}$)", 
         va='center', rotation='vertical', fontsize=22)

fig.text(0.41, 0.075, "Genome Position (Mbp)", 
         va='center', rotation='horizontal', fontsize=22)

## fig.savefig("ATCC_Env_test_plot.tiff", bbox_inches='tight')

plt.show()